In [0]:
import sklearn.model_selection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy
import sys

data=pd.read_csv("/content/drive/My Drive/Project_dataset/creditcard.csv")

In [0]:
data.hist(figsize=(20,20))
plt.show()  

In [0]:
#no of valid and fraudlent transcation
fraud=data[data["Class"]==1]
valid=data[data["Class"]==0]
outlier_fraction=len(fraud)/float(len(valid))

In [0]:
#correleation matrix
corr=data.corr()
fig=plt.figure(figsize=(12,9))
sns.heatmap(corr,vmax=0.8,square=True)
plt.show()

In [0]:
#format dataset
columns=data.columns.to_list()
#remove the class field
columns=[c for c in columns if c not in ["Class"]]
#prediction on variable
target="Class"
x=data[columns]
y=data[target]

In [0]:
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
#define random_state 
state=1
#define classifierfor outlier detection
classifier={
    "Isolation Forest": IsolationForest(max_samples=len(x),contamination=outlier_fraction,random_state=state),
    "Local Outlier Factor": LocalOutlierFactor(n_neighbors=20,contamination=outlier_fraction)
}

In [0]:
#fit the model
n_outlier=len(fraud)
for i , (clf_name, clf) in enumerate(classifier.items()):
  #fit data and tag outlier
  #if localoutlier factor it will fir_predict if isolationforest it will fir then predict
  if clf_name=="Local Outlier Factor":
    y_pred=clf.fit_predict(x)
    scores_pred=clf.negative_outlier_factor_
  else:
    clf.fit(x)
    scores_pred=clf.decision_function(x)
    y_pred=clf.predict(x)

    #reshape prediction 0 for valid and 1 for fraud
    y_pred[y_pred==1]=0
    y_pred[y_pred==-1]=1

    n_error=(y_pred!=y).sum()
    print('{}:{}'.format(clf_name,n_error))
    print(accuracy_score(y,y_pred))
    print(classification_report(y,y_pred))    